In [ ]:
class Parser:
    def __init__(self, filename):
        with open(filename, 'r') as f:
            self.lines = f.readlines()
        self.index = 0
        self.commands = list()
        
        for line in self.lines:
            processed_line = line.strip().replace(' ', '').split('//')[0]
            if processed_line:
                self.commands.append(processed_line)

        self.current_line = None

    def has_more_commands(self):
        return self.index < len(self.commands)
    
    def advance(self):
        if self.has_more_commands():
            self.current_line = self.commands[self.index]
            self.index += 1

    def command_type(self):
        if self.current_line.startswith('@'):
            return 'A_COMMAND'
        elif any(char in self.current_line for char in ['=', ';']):
            return 'C_COMMAND'
        else:
            return 'L_COMMAND'
        
    def symbol(self):
        if self.command_type() == 'A_COMMAND':
            return self.current_line.strip('@')
        elif self.command_type() == 'L_COMMAND':
            return self.current_line.strip('()')
        
    def dest(self):
        if self.command_type() == 'C_COMMAND':
            if '=' in self.current_line:
                return self.current_line.split('=')[0]
            else:
                return None
            
    def comp(self):
        if self.command_type() == 'C_COMMAND':
            if ';' and '=' in self.current_line:
                return self.current_line.split('=')[1].split(';')[0]
            elif '=' in self.current_line:
                return self.current_line.split('=')[1]
            elif ';' in self.current_line:
                return self.current_line.split(';')[0]
            else:
                return self.current_line
            
    def jump(self):
        if self.command_type() == 'C_COMMAND':
            if ';' in self.current_line:
                return self.current_line.split(';')[1]
            else:
                return None
            
    def reset(self):
        self.index = 0

In [270]:
class Code:
    def __init__(self):
        self.dest_map = {
            None: '000',
            'M': '001',
            'D': '010',
            'MD': '011',
            'A': '100',
            'AM': '101',
            'AD': '110',
            'AMD': '111'
        }

        self.comp_map = {
            '0': '0101010',
            '1': '0111111',
            '-1': '0111010',
            'D': '0001100',
            'A': '0110000',
            'M': '1110000',
            '!D': '0001101',
            '!A': '0110001',
            '!M': '1110001',
            '-D': '0001111',
            '-A': '0110011',
            '-M': '1110011',
            'D+1': '0011111',
            'A+1': '0110111',
            'M+1': '1110111',
            'D-1': '0001110',
            'A-1': '0110010',
            'M-1': '1110010',
            'D+A': '0000010',
            'D+M': '1000010',
            'D-A': '0010011',
            'D-M': '1010011',
            'A-D': '0000111',
            'M-D': '1000111',
            'D&A': '0000000',
            'D&M': '1000000',
            'D|A': '0010101',
            'D|M': '1010101'
        }

        self.jump_map = {
            None: '000',
            'JGT': '001',
            'JEQ': '010',
            'JGE': '011',
            'JLT': '100',
            'JNE': '101',
            'JLE': '110',
            'JMP': '111'
        }

    def dest(self, symbol):
        return self.dest_map[symbol]
    
    def comp(self, symbol):
        return self.comp_map[symbol]
    
    def jump(self, symbol):
        return self.jump_map[symbol]


In [271]:
class SymbolTable():
    def __init__(self):
        self.table = {
            'R0': 0,
            'R1': 1,
            'R2': 2,
            'R3': 3,
            'R4': 4,
            'R5': 5,
            'R6': 6,
            'R7': 7,
            'R8': 8,
            'R9': 9,
            'R10': 10,
            'R11': 11,
            'R12': 12,
            'R13': 13,
            'R14': 14,
            'R15': 15,
            'SCREEN': 16384,
            'KBD': 24576,
            'SP': 0,
            'LCL': 1,
            'ARG': 2,
            'THIS': 3,
            'THAT': 4
        }
        
    def add_entry(self, symbol, address):
        self.table[symbol] = address

    def contains(self, symbol):
        return symbol in self.table
    
    def get_address(self, symbol):
        return self.table[symbol]


In [220]:
parser = Parser('Max.asm')
coder = Code()
symbol_table = SymbolTable()

In [259]:
parser.advance()

In [260]:
parser.current_line

'0;JMP'

In [186]:
parser.symbol()

In [ ]:
parser.symbol().isnumeric()

In [200]:
parser.comp()

['D', 'JGT']

In [164]:
if parser.symbol().isnumeric():
    decimal_value = parser.symbol()
else:
    if symbol_table.contains(parser.symbol()):
        decimal_value = symbol_table.get_address(parser.symbol())
    else:
        symbol_table.add_entry(parser.symbol(), 16)

binary_value = format(int(decimal_value), '015b')
machine_instruction = f'0{binary_value}'    # 0 = opcode, 15-bit address
print(machine_instruction)

0000000000000000


In [319]:
parser = Parser('Pong.asm')
coder = Code()
symbol_table = SymbolTable()

instruction_address = 0
while parser.has_more_commands():
    parser.advance()
    if parser.command_type() == 'L_COMMAND':
        symbol_table.add_entry(parser.symbol(), instruction_address)    # The instruction after the label declaration is where the label block starts
    else:
        instruction_address += 1    # label declarations are not real instructions and should thus not be considered when we track instruction addresses

parser.reset()

machine_instructions = list()
memory_idx = 16     # memory starting at register 16 is dedicated for variables, increase memory index by one every time a new variable is created

while parser.has_more_commands():
    parser.advance()
    
    print(parser.current_line)

    if parser.command_type() == 'C_COMMAND':
        dest_field = coder.dest(parser.dest())
        comp_field = coder.comp(parser.comp())
        jump_field = coder.jump(parser.jump())
        machine_instruction = f'111{comp_field}{dest_field}{jump_field}'

        machine_instructions.append(machine_instruction)
        print(machine_instruction)

    elif parser.command_type() == 'A_COMMAND':
        if parser.symbol().isnumeric():
            decimal_value = parser.symbol()
        else:
            if symbol_table.contains(parser.symbol()):
                decimal_value = symbol_table.get_address(parser.symbol())
            else:
                symbol_table.add_entry(parser.symbol(), memory_idx)
                decimal_value = memory_idx
                memory_idx += 1

        binary_value = format(int(decimal_value), '015b')
        machine_instruction = f'0{binary_value}'    # 0 = opcode, 15-bit address

        machine_instructions.append(machine_instruction)
        print(machine_instruction)

with open('Pong.hack', 'w') as file:
    file.write('\n'.join(machine_instructions))   

@256
0000000100000000
D=A
1110110000010000
@SP
0000000000000000
M=D
1110001100001000
@133
0000000010000101
0;JMP
1110101010000111
@R15
0000000000001111
M=D
1110001100001000
@SP
0000000000000000
AM=M-1
1111110010101000
D=M
1111110000010000
A=A-1
1110110010100000
D=M-D
1111000111010000
M=0
1110101010001000
@END_EQ
0000000000010011
D;JNE
1110001100000101
@SP
0000000000000000
A=M-1
1111110010100000
M=-1
1110111010001000
(END_EQ)
@R15
0000000000001111
A=M
1111110000100000
0;JMP
1110101010000111
@R15
0000000000001111
M=D
1110001100001000
@SP
0000000000000000
AM=M-1
1111110010101000
D=M
1111110000010000
A=A-1
1110110010100000
D=M-D
1111000111010000
M=0
1110101010001000
@END_GT
0000000000100011
D;JLE
1110001100000110
@SP
0000000000000000
A=M-1
1111110010100000
M=-1
1110111010001000
(END_GT)
@R15
0000000000001111
A=M
1111110000100000
0;JMP
1110101010000111
@R15
0000000000001111
M=D
1110001100001000
@SP
0000000000000000
AM=M-1
1111110010101000
D=M
1111110000010000
A=A-1
1110110010100000
D=M-D
11

In [ ]:
parser = Parser('Max.asm')
coder = Code()

machine_instructions = list()

while parser.has_more_commands():
    parser.advance()
    if parser.command_type() == 'L_COMMAND':
        if symbol_table.contains(parser.symbol()):
            decimal_value = symbol_table.get_address(parser.symbol())
        else:
            symbol_table.add_entry(parser.symbol(), print(parser.index) + 1)    # The line after the label is where the label block starts

parser.reset()

while parser.has_more_commands():
    parser.advance()
    memory_idx = 16     # memory starting at register 16 is dedicated for variables, increase memory index by one every time a new variable is created
    print(parser.current_line)

    if parser.command_type() == 'C_COMMAND':
        dest_field = coder.dest(parser.dest())
        comp_field = coder.comp(parser.comp())
        jump_field = coder.jump(parser.jump())
        machine_instruction = f'111{comp_field}{dest_field}{jump_field}'

    elif parser.command_type() == 'A_COMMAND':
        if parser.symbol().isnumeric():
            decimal_value = parser.symbol()
        else:
            if symbol_table.contains(parser.symbol()):
                decimal_value = symbol_table.get_address(parser.symbol())
            else:
                symbol_table.add_entry(parser.symbol(), memory_idx)
                memory_idx += 1

        binary_value = format(int(decimal_value), '015b')
        machine_instruction = f'0{binary_value}'    # 0 = opcode, 15-bit address
    print(machine_instruction)

    machine_instructions.append(machine_instruction)

with open('Add.hack', 'w') as file:
    file.write('\n'.join(machine_instructions))    

In [ ]:
parser = Parser('Add.asm')
coder = Code()

machine_instructions = list()

while parser.has_more_commands():
    parser.advance()

    if parser.command_type() in ['C_COMMAND']:
        dest_field = coder.dest(parser.dest())
        comp_field = coder.comp(parser.comp())
        jump_field = coder.jump(parser.jump())
        machine_instruction = f'111{comp_field}{dest_field}{jump_field}'

    elif parser.command_type() in ['A_COMMAND']:
        decimal_value = parser.symbol()
        binary_value = format(int(decimal_value), '015b')
        machine_instruction = f'0{binary_value}'    # 0 = opcode, 15-bit address


    machine_instructions.append(machine_instruction)

with open('Add.hack', 'w') as file:
    file.write('\n'.join(machine_instructions))        